# Modelos y entrenamiento

In [1]:
# Librerías esenciales
import os
import pandas as pd
import numpy as np

# Modelos y evaluación
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter




# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de gráficos
sns.set_theme(style="whitegrid")
# import optuna
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score

print("Librerías importadas correctamente.")


✅ Librerías importadas correctamente.


In [2]:
# Definir rutas de los datasets procesados
train_path = "../data/df_train_processed.feather"
test_path = "../data/df_test_processed.feather"

# Cargar los datasets
df_train = pd.read_feather(train_path)
df_test = pd.read_feather(test_path)

# Mostrar las dimensiones de los datos cargados
print(f"Datos cargados correctamente.")
print(f"Dimensiones - Train: {df_train.shape}, Test: {df_test.shape}")

Datos cargados correctamente.
Dimensiones - Train: (652515, 14), Test: (569886, 13)


In [3]:
df_train.info()
df_test.info()  

<class 'pandas.core.frame.DataFrame'>
Index: 652515 entries, 0 to 699999
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   trip_id              652515 non-null  int64   
 1   duration             652515 non-null  int64   
 2   start_lat            652515 non-null  float64 
 3   start_lon            652515 non-null  float64 
 4   end_lat              652515 non-null  float64 
 5   end_lon              652515 non-null  float64 
 6   trip_route_category  652515 non-null  category
 7   passholder_type      652515 non-null  category
 8   start_station        652515 non-null  category
 9   end_station          652515 non-null  category
 10  hour                 652515 non-null  int32   
 11  day_of_week          652515 non-null  category
 12  year_month           652515 non-null  object  
 13  year                 652515 non-null  int32   
dtypes: category(5), float64(4), int32(2), int64(2), object(1)

In [4]:
# Al guardarlos en formato feather hubo un detalle con datos object, por lo que se deben convertir a categoricos
df_train["year_month"] = df_train["year_month"].astype("category")
df_test["year_month"] = df_test["year_month"].astype("category")
df_train.info()
df_test.info()  

<class 'pandas.core.frame.DataFrame'>
Index: 652515 entries, 0 to 699999
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   trip_id              652515 non-null  int64   
 1   duration             652515 non-null  int64   
 2   start_lat            652515 non-null  float64 
 3   start_lon            652515 non-null  float64 
 4   end_lat              652515 non-null  float64 
 5   end_lon              652515 non-null  float64 
 6   trip_route_category  652515 non-null  category
 7   passholder_type      652515 non-null  category
 8   start_station        652515 non-null  category
 9   end_station          652515 non-null  category
 10  hour                 652515 non-null  int32   
 11  day_of_week          652515 non-null  category
 12  year_month           652515 non-null  category
 13  year                 652515 non-null  int32   
dtypes: category(6), float64(4), int32(2), int64(2)
memory usa

## Entrenamiento de modelos

Necesitamos dividir los datos en tres conjuntos:
1. **Train (70%)** Se usa para entrenar el modelo.  
2. **Validation (15%)**  Se usa para ajustar hiperparámetros y evaluar el desempeño durante el entrenamiento.  
3. **Test (15%)** Se usa para evaluar el rendimiento final antes de usar el modelo en producción.  


In [5]:
from sklearn.model_selection import train_test_split

# Definir variables predictoras y objetivo
X = df_train.drop(columns=["passholder_type"])  # Todas menos la variable objetivo
y = df_train["passholder_type"]  # Variable a predecir

# Primero separamos 70% train y 30% restante
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Ahora dividimos el 30% restante en validación (15%) y test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

# Verificar dimensiones
print(f"Dimensiones después del split:")
print(f"Train: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Validation: X_val {X_val.shape}, y_val {y_val.shape}")
print(f"Test: X_test {X_test.shape}, y_test {y_test.shape}")

Dimensiones después del split:
Train: X_train (456760, 13), y_train (456760,)
Validation: X_val (97877, 13), y_val (97877,)
Test: X_test (97878, 13), y_test (97878,)


In [6]:
# Inicializar el LabelEncoder y ajustar con los valores de la variable objetivo
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convertir `y_train`, `y_val`, y `y_test` en NumPy arrays para evitar errores
y_train = y_train_encoded
y_val = y_val_encoded
y_test = y_test_encoded

# Verificar las clases codificadas
print("Clases codificadas:", label_encoder.classes_)

Clases codificadas: ['Annual Pass' 'Flex Pass' 'Monthly Pass' 'One Day Pass' 'Testing'
 'Walk-up']


### Modelos

In [9]:
# Convertir los datasets a formato LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

# Definir hiperparámetros del modelo
params = {
    "objective": "multiclass",
    "num_class": len(label_encoder.classes_),
    "boosting_type": "gbdt",
    "metric": "multi_logloss",
    "learning_rate": 0.07,
    "max_depth": 13,
    "num_leaves": 43,
    "min_data_in_leaf": 30,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "random_state": 42,
    "verbose": -1
}

# Entrenar modelo sin `early_stopping_rounds`
print("Entrenando LightGBM...")
lgb_model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=500
)

# Predicciones en validación
y_val_pred_proba = lgb_model.predict(X_val)

# Convertir probabilidades a clases
y_val_pred = y_val_pred_proba.argmax(axis=1)

# Evaluar desempeño
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy en validación: {accuracy:.4f}")

print("\nReporte de Clasificación:")
print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))





Entrenando LightGBM...
Accuracy en validación: 0.7419

Reporte de Clasificación:
              precision    recall  f1-score   support

 Annual Pass       0.69      0.37      0.49      4941
   Flex Pass       0.64      0.13      0.21      1625
Monthly Pass       0.77      0.90      0.83     55853
One Day Pass       0.52      0.17      0.26      5904
     Testing       0.06      0.14      0.08         7
     Walk-up       0.70      0.65      0.68     29547

    accuracy                           0.74     97877
   macro avg       0.56      0.39      0.42     97877
weighted avg       0.73      0.74      0.72     97877



El modelo LightGBM tiene un **accuracy de 74.14%**, lo cual no está mal, pero hay puntos clave que podemos mejorar

1. **`Monthly Pass`** es la clase mejor predicha con **f1-score de 0.83**, lo que indica que el modelo captura bien su patrón.
2. **`Annual Pass`, `Flex Pass` y `One Day Pass`** tienen baja precisión y recall, lo que sugiere que el modelo tiene problemas diferenciándolos.
3. **`Testing`** es completamente ignorada con un f1-score de **0.00**, lo que indica que el modelo no encuentra ejemplos suficientes para aprender de ella y es logico porque había muy pocos datos, quizá haciendo un balanceo de clases se podría mejorar.
4. **`Walk-up`** tiene un desempeño aceptable pero no óptimo.




In [11]:
import pickle

# Guardar el modelo LightGBM
with open("lightgbm_model.pkl", "wb") as f:
    pickle.dump(lgb_model, f)



print("Modelos guardados exitosamente")



✅ Modelos guardados exitosamente


Entiendo, la estructura de la **submisión** debe contener dos columnas:

1. `trip_id`: Identificador único del viaje.
2. `passholder_type`: Predicción del tipo de pase (`Monthly Pass`, `Walk-up`, etc.).

---

## **📌 Estrategia para construir la submisión correctamente**
### **1️⃣ Transformaciones necesarias antes del modelado**
- **Asegurar que `trip_id` esté presente en el dataset de test**.
- **Asegurar que `passholder_type` es la variable objetivo en train**.
- **Asegurar consistencia en los tipos de datos entre `df_train` y `df_test`**.
- **Generar características en `df_test` que existen en `df_train` pero faltan en test**:
  - `hour`, `day_of_week`, `year_month`, `year`.

---

### **2️⃣ Modelos candidatos**
Dado que el objetivo es **clasificación multiclase** (`passholder_type` tiene varias categorías), podemos considerar modelos eficientes y livianos para evitar un peso excesivo:

| Modelo | Pros | Contras |
|--------|------|---------|
| **Random Forest** | Bueno con categóricas, fácil de interpretar | Puede ser pesado si tiene muchas categorías distintas |
| **XGBoost** | Eficiente y optimizado para tabulares | Puede ser más lento que otros modelos ligeros |
| **LightGBM** | Rápido y eficiente con datos categóricos | Puede ser sensible a datos desbalanceados |
| **Regresión Logística** | Sencillo, interpretativo y liviano | Puede no capturar relaciones no lineales |
| **Naive Bayes** | Muy rápido en inferencia | Suponiendo independencia de features, lo cual puede no ser realista |




In [12]:
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569886 entries, 0 to 569885
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   trip_id              569886 non-null  int64   
 1   duration             569886 non-null  int64   
 2   start_lat            569886 non-null  float64 
 3   start_lon            569886 non-null  float64 
 4   end_lat              569886 non-null  float64 
 5   end_lon              569886 non-null  float64 
 6   trip_route_category  569886 non-null  category
 7   start_station        569886 non-null  category
 8   end_station          569886 non-null  category
 9   hour                 569886 non-null  int32   
 10  day_of_week          569886 non-null  category
 11  year_month           569886 non-null  category
 12  year                 569886 non-null  int32   
dtypes: category(5), float64(4), int32(2), int64(2)
memory usage: 34.3 MB


In [18]:
import pickle

# Cargar el modelo LightGBM
with open("lightgbm_model.pkl", "rb") as f:
    lgb_model = pickle.load(f)

print("Modelos cargados exitosamente.")


✅ Modelos cargados exitosamente.


In [24]:
# Asegurar que las variables categóricas en df_test tienen las mismas categorías que X_train
categorical_columns = ["trip_route_category", "start_station", "end_station", "day_of_week", "year_month"]

for col in categorical_columns:
    df_test[col] = df_test[col].astype("category")
    df_test[col] = df_test[col].cat.set_categories(X_train[col].cat.categories)

print("Se han corregido las categorías en df_test para coincidir con X_train.")



✅ Se han corregido las categorías en df_test para coincidir con X_train.


In [25]:

# Hacer predicción asegurándonos de que df_test solo tenga las columnas esperadas
X_test_model = df_test[X_train.columns]  # Solo seleccionamos las columnas usadas en entrenamiento

# Hacer la predicción
y_test_pred_proba = lgb_model.predict(X_test_model)

# Convertir probabilidades a etiquetas
y_test_pred = y_test_pred_proba.argmax(axis=1)

# Convertir etiquetas numéricas a nombres de categorías
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)

# Crear el DataFrame final con trip_id y la predicción
df_submission = pd.DataFrame({
    "trip_id": df_test["trip_id"],
    "passholder_type": y_test_pred_labels
})

# Guardar en CSV
df_submission.to_csv("submission_lightgbm.csv", index=False)

print("Predicciones guardadas en 'submission_lightgbm.csv'")



✅ Predicciones guardadas en 'submission_lightgbm.csv'
